In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [5]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [7]:
fitted_logreg_model = my_log_reg_model.fit(my_data)

In [8]:
log_summary = fitted_logreg_model.summary

In [9]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [11]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])

In [12]:
final_model = LogisticRegression()

In [13]:
fit_final = final_model.fit(lr_train)

In [14]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [15]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[17.2563824634318...|[0.99999996796326...|       0.0|
|  0.0|(692,[124,125,126...|[25.5081950680507...|[0.99999999999164...|       0.0|
|  0.0|(692,[124,125,126...|[33.6395778966693...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[25.1579130479254...|[0.99999999998814...|       0.0|
|  0.0|(692,[125,126,127...|[35.5484036050341...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[22.8423655099664...|[0.99999999987986...|       0.0|
|  0.0|(692,[126,127,128...|[31.6824870582042...|[0.99999999999998...|       0.0|
|  0.0|(692,[127,128,129...|[22.7730160084116...|[0.99999999987123...|       0.0|
|  0.0|(692,[150,151,152...|[16.5387424237372...|[0.99999993433780...|       0.0|
|  0.0|(692,[152

In [16]:
prediction_and_labels.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              label|        prediction|
+-------+-------------------+------------------+
|  count|                 30|                30|
|   mean| 0.6333333333333333|0.5666666666666667|
| stddev|0.49013251785356093|0.5040069329937309|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [17]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,MultilabelClassificationEvaluator)

In [18]:
my_eval = BinaryClassificationEvaluator()

In [19]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [20]:
my_final_roc

1.0